In [1]:
# Import necessary packages
import ee
import geemap


In [2]:
# Initiate interactive map
Map = geemap.Map(center=(40, -100), zoom=4)
Map.add_basemap('ROADMAP') # Add Google Map
Map

layer already on map: TileLayer(attribution='Google', name='Google Maps', options=['attribution', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms'], url='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}')
Basemap can only be one of the following:
  ROADMAP
  SATELLITE
  TERRAIN
  HYBRID
  ESRI
  Esri Ocean
  Esri Satellite
  Esri Standard
  Esri Terrain
  Esri Transportation
  Esri Topo World
  Esri National Geographic
  Esri Shaded Relief
  Esri Physical Map
  FWS NWI Wetlands
  FWS NWI Wetlands Raster
  Google Maps
  Google Satellite
  Google Terrain
  Google Satellite Hybrid
  NLCD 2016 CONUS Land Cover
  NLCD 2013 CONUS Land Cover
  NLCD 2011 CONUS Land Cover
  NLCD 2008 CONUS Land Cover
  NLCD 2006 CONUS Land Cover
  NLCD 2004 CONUS Land Cover
  NLCD 2001 CONUS Land Cover
  USGS NAIP Imagery
  USGS Hydrography
  USGS 3DEP Elevation
  OpenStreetMap.Mapnik
  OpenStreetMap.BlackAndWhite
  OpenStreetMap.DE
  OpenStreetMap.F

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [4]:
# Get raw data input for simple composite
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')

# Filter collection for by date
collection = l8.filterDate('2015-1-1', '2018-7-1')

# Create cloud-free composite with default params
composite = ee.Algorithms.Landsat.simpleComposite(collection)

# Create cloud-free composite with custom params
# customComposite = ee.Algorithms.Landsat.simpleComposite(
#     collection, percentile=75, cloudScoreRange=5)

# What's the difference in the syntax here?  What do the asterisks do?
customComposite = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': collection,
    'percentile': 75,
    'cloudScoreRange': 5})

# Pick a spot with a lot of clouds to test composite
Map.setCenter(-47.6735, -0.6344, 12)

# Display the two composites as RGB
Map.addLayer(
    composite, {'bands': ['B4', 'B3', 'B2'], 'max': 128}, 'TOA composite')
Map.addLayer(customComposite, {
             'bands': ['B4', 'B3', 'B2'], 'max': 128}, 'TOA Custom Composite')


# Also display the composite with the GEE docs viz params
Map.addLayer(composite, {'bands': ['B4', 'B3', 'B2'], 
                         'max': [0.3, 0.4, 0.3]}, 'TOA composite, GEE docs')